In [1]:
from utilities import *
from config import *
from data_preparing import *
from DNN_model import *
#import tsfresh
from tsfresh.feature_extraction import MinimalFCParameters,EfficientFCParameters
from tsfresh import feature_extraction
from tsfresh import extract_features
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.decomposition import PCA
import tsfresh
from tsfresh import feature_selection
from tsfresh import extract_features, select_features

In [2]:
attribute = ['Unit','Time','T24','T30','T50','P30','Nc','Ps30','phi','BPR']
df_train = pd.read_csv('./Dataset 4/preprocessed data/train_data.csv',
                      names = attribute, header = None) 
df_test = pd.read_csv('./Dataset 4/preprocessed data/test_data.csv',
                      names = attribute, header = None) 

train_label=pd.read_csv('./Dataset 4/preprocessed data/train_label.csv', header=None).values
test_label=pd.read_csv('./Dataset 4/preprocessed data/test_label.csv', header=None).values

train_dataset=AircraftDataset(df_train,train_label,attribute[2:]) # 不插0计算创建dataset的子类
test_dataset = AircraftDataset(df_test,test_label,attribute[2:])

In [3]:
train_dataset[0]

{'input': tensor([[ 1.5672,  0.3100,  0.4226,  ...,  0.4308,  0.0135, -0.6465],
         [ 0.6342,  0.6206,  0.3720,  ...,  0.2849, -0.4610, -0.6520],
         [ 1.1697,  0.2749,  0.6845,  ...,  0.3607, -0.1670,  0.0225],
         ...,
         [ 1.7043,  1.5343,  1.3453,  ...,  1.7412,  1.7486, -1.5320],
         [ 1.4320,  1.5611,  1.5431,  ...,  1.6309,  1.8111, -1.5760],
         [ 1.5806,  1.6450,  1.6684,  ...,  1.5178,  1.8335, -1.5472]]),
 'lifetime': tensor(321),
 'mode': tensor([0., 1.])}

In [4]:
train_dataset_expend=AircraftDataset_expend(df_train,train_label,False,attribute[2:]) # 构建切割后的训练集

In [5]:
train_dataset_expend[0]

{'input': tensor([[ 1.5672e+00,  3.1000e-01,  4.2264e-01, -5.1663e-01,  1.5127e-01,
           4.3085e-01,  1.3544e-02, -6.4655e-01],
         [ 6.3418e-01,  6.2057e-01,  3.7201e-01, -4.4173e-01,  1.4967e-01,
           2.8491e-01, -4.6101e-01, -6.5199e-01],
         [ 1.1697e+00,  2.7492e-01,  6.8451e-01,  3.2798e-01, -9.0540e-02,
           3.6068e-01, -1.6697e-01,  2.2482e-02],
         [ 9.8675e-01,  3.0927e-02,  7.3786e-01,  1.7821e-01,  1.9794e-01,
           4.0800e-01,  3.3979e-02, -6.2156e-01],
         [ 9.9437e-01,  5.9467e-01,  4.3614e-02,  1.8411e-01, -5.1194e-01,
           6.3355e-01,  2.6670e-01, -8.3924e-01],
         [ 1.0009e+00, -9.2899e-02,  4.1204e-01,  2.7692e-01,  1.3602e-01,
           1.5384e-01, -1.0909e-01, -2.3631e-01],
         [ 1.1190e+00,  6.9071e-01,  4.9501e-01,  2.9445e-01,  3.1170e-01,
           5.2719e-02,  2.8390e-01, -6.9499e-01],
         [ 1.2344e+00,  3.8804e-01,  2.5192e-01, -9.8222e-01, -5.6276e-02,
           2.0359e-01, -1.3055e-01, -3.59

In [6]:
seed_everything(CFG.seed + 100)
# idx_list=[np.random.randint(1,19720) for i in range(200)]
idx_list = np.random.choice(range(48799), 250, replace=False)
idx_list.sort()
#从训练集中抽取样本
dataframe_list=[]
for idx in idx_list:
    dataframe_list.append(get_dataframe_ofcuttedsequence_new(train_dataset_expend,
                                                             idx,attribute[2:])) # len is 48799
train_dataset_expend_dataframe=pd.concat(dataframe_list).reset_index(drop=True)
train_dataset_expend_dataframe

,T24,T30,T50,P30,Nc,Ps30,phi,BPR,Unit,Time
0,1.567173,0.309998,0.422641,-0.516634,0.151267,0.430850,0.013544,-0.646547,49,1
1,0.634182,0.620573,0.372009,-0.441732,0.149668,0.284912,-0.461014,-0.651987,49,2
2,1.169667,0.274920,0.684506,0.327982,-0.090540,0.360682,-0.166967,0.022482,49,3
3,0.986752,0.030927,0.737858,0.178214,0.197943,0.408003,0.033979,-0.621558,49,4
4,0.994371,0.594673,0.043614,0.184108,-0.511944,0.633553,0.266701,-0.839238,49,5
...,...,...,...,...,...,...,...,...,...,...
42241,0.754549,1.320966,1.159233,1.518283,1.244740,1.168130,1.523827,-1.125358,48441,187
42242,1.232257,1.479147,0.917902,1.466052,1.267992,1.078741,1.533927,-1.191995,48441,188
42243,0.788507,1.469536,1.273774,1.342655,1.211261,1.154032,1.341925,-1.233431,48441,189
42244,0.817451,1.383269,1.098976,1.490071,1.358783,1.315601,1.465251,-1.348093,48441,190


In [7]:
# 原始RUL & mode
RUL = pd.read_csv('./Dataset 4/preprocessed data/RUL.csv',
                      names = None, header = None).values
#切割后提取的部分训练集的RUL
mode_list=[]
for  idx in idx_list:
    if train_dataset_expend[idx]["mode"][0].item() == 0:
        mode_list.append(0)
    elif train_dataset_expend[idx]["mode"][0].item() == 1:
        mode_list.append(1)
mode=pd.Series(mode_list)
mode.index+=1
RUL_list=[train_dataset_expend[idx]["RUL"].item() for idx in idx_list]
RUL_y=pd.Series(RUL_list)
RUL_y.index+=1

target_a=RUL_y.values
target_b=mode.values

In [8]:
train_extracted_features = extract_features(train_dataset_expend_dataframe,
                                      column_id="Unit",
                                        column_sort="Time",
                                         default_fc_parameters =EfficientFCParameters())

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 40/40 [00:28<00:00,  1.41it/s]


In [9]:
feature_raw=train_extracted_features.copy().reset_index(drop=True)
feature_raw.index+=1

In [10]:
p_values=tsfresh.feature_selection.relevance.calculate_relevance_table(impute(feature_raw), 
                                                                       RUL_y,ml_task="regression")
p_values

C:\ProgramData\Anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['phi__query_similarity_count__query_None__threshold_0.0'
 'BPR__query_similarity_count__query_None__threshold_0.0'
 'T24__query_similarity_count__query_None__threshold_0.0'
 'T30__query_similarity_count__query_None__threshold_0.0'
 'T50__query_similarity_count__query_None__threshold_0.0'
 'P30__query_similarity_count__query_None__threshold_0.0'
 'Nc__query_similarity_count__query_None__threshold_0.0'
 'Ps30__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


,feature,type,p_value,relevant
feature,,,,
"Ps30__linear_trend__attr_""rvalue""","Ps30__linear_trend__attr_""rvalue""",real,8.974965e-59,True
"T50__linear_trend__attr_""rvalue""","T50__linear_trend__attr_""rvalue""",real,1.641197e-57,True
"Ps30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","Ps30__agg_linear_trend__attr_""rvalue""__chunk_l...",real,2.115772e-55,True
"T50__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","T50__agg_linear_trend__attr_""rvalue""__chunk_le...",real,8.349195e-53,True
"Ps30__agg_autocorrelation__f_agg_""mean""__maxlag_40","Ps30__agg_autocorrelation__f_agg_""mean""__maxla...",real,8.952126e-53,True
...,...,...,...,...
Ps30__value_count__value_1,Ps30__value_count__value_1,constant,NaN,False
Ps30__value_count__value_-1,Ps30__value_count__value_-1,constant,NaN,False
Ps30__ratio_beyond_r_sigma__r_7,Ps30__ratio_beyond_r_sigma__r_7,constant,NaN,False


In [11]:
p_values_=p_values[abs(p_values["p_value"])<1e-15].reset_index(drop=True)
#p_values_
print(len(p_values_['feature'].values))
feature_test=feature_raw.loc[:,p_values_['feature'].values].values
feature_a=feature_test[:,0]
feature_b=feature_test[:,1]
feature_c=feature_test[:,-1]

666


In [12]:
p_values_mode=tsfresh.feature_selection.relevance.calculate_relevance_table(impute(feature_raw), 
                                                                       mode,ml_task='classification', 
                                                                            #fdr_level=1.105
                                                                           )

In [13]:
p_values_mode_=p_values_mode[abs(p_values_mode["p_value"])<1e-15].reset_index(drop=True)
len(p_values_mode_["feature"].values)
feature_test_mode=feature_raw.loc[:,p_values_mode_["feature"].values].values
feature_d=feature_test_mode[:,0]
feature_e=feature_test_mode[:,1]
feature_f=feature_test_mode[:,-1]

In [14]:
# 组间方差
range_1=np.intersect1d(np.where(target_a<=100),np.where(target_b==1))
range_2=np.intersect1d(np.where(target_a<=100),np.where(target_b==0))
range_3=np.intersect1d(np.where(target_a>100),np.where(target_b==0))
range_4=np.intersect1d(np.where(target_a>100),np.where(target_b==1))
range_list=[range_1,range_2,range_3,range_4]

In [15]:
# 构建tsfresh需要的filter
selected_columns=np.hstack([p_values_["feature"].values,p_values_mode_["feature"].values])
g=feature_raw.copy()
X_tsfresh_filter=g[selected_columns]
X_tsfresh_filter

,"Ps30__linear_trend__attr_""rvalue""","T50__linear_trend__attr_""rvalue""","Ps30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","T50__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","Ps30__agg_autocorrelation__f_agg_""mean""__maxlag_40","Ps30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""min""","T30__linear_trend__attr_""rvalue""","T30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","T50__agg_autocorrelation__f_agg_""mean""__maxlag_40","Ps30__linear_trend__attr_""pvalue""",...,"phi__fft_coefficient__attr_""abs""__coeff_1","Ps30__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",T24__range_count__max_1__min_-1,"BPR__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""",phi__time_reversal_asymmetry_statistic__lag_1,phi__autocorrelation__lag_9,"P30__fft_coefficient__attr_""abs""__coeff_1","phi__fft_coefficient__attr_""imag""__coeff_8","P30__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","phi__fft_coefficient__attr_""angle""__coeff_2"
1,0.036808,-0.045074,0.085754,-0.125201,-0.015258,-0.037778,-0.021656,-0.033845,-0.015560,7.189728e-01,...,5.241730,0.318827,20.0,-1.000000,-0.002560,0.132959,5.073326,0.433811,0.033710,131.614472
2,0.678206,0.593719,0.885443,0.876356,0.384595,0.830396,0.427227,0.727215,0.274177,5.870719e-34,...,5.335814,0.201401,178.0,0.911804,0.001011,-0.004809,7.133236,0.553403,-0.029456,-97.403751
3,0.391805,0.377902,0.648412,0.732115,0.120158,0.481087,0.298577,0.651584,0.101254,1.593747e-09,...,13.604693,0.270685,148.0,-0.778237,0.005593,0.435131,11.571244,3.153015,-0.022754,93.630000
4,0.512313,0.468434,0.752620,0.794643,0.226464,0.598739,0.377685,0.723493,0.173509,7.293960e-18,...,21.041474,0.272041,158.0,-0.825524,0.010225,0.568850,17.928545,3.755927,-0.022754,86.975328
5,-0.065119,-0.071720,-0.132862,-0.110864,-0.024150,-0.177193,-0.123445,-0.267598,-0.013678,5.977829e-01,...,1.695958,0.220908,46.0,1.000000,-0.000987,0.066453,1.017173,1.802740,0.939187,-25.497923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.540671,0.441477,0.848319,0.773979,0.241943,0.682978,0.256907,0.518524,0.136777,4.805643e-16,...,3.491504,0.116001,100.0,0.964714,0.002099,0.053640,2.611411,-1.931425,0.083199,-94.004008
247,0.420526,0.361882,0.745867,0.714166,0.084149,0.615571,0.291753,0.577552,0.063121,5.294984e-06,...,0.819272,0.261161,97.0,0.997821,-0.001705,0.035548,1.297283,0.124883,-0.184167,-41.988527
248,0.816956,0.730554,0.942471,0.924612,0.520916,0.879056,0.645477,0.893074,0.388455,1.545047e-47,...,6.822747,0.260225,130.0,0.954109,-0.004147,0.264636,9.300413,-0.466840,-0.009180,-78.375580
249,0.153639,0.129724,0.273206,0.247850,-0.023117,0.308664,0.087300,0.190126,-0.015016,2.217412e-01,...,4.269818,0.389223,62.0,1.000000,0.001498,-0.008270,2.060984,-0.589316,0.373222,-10.918410


In [17]:
su_n_n_list=[]
su_n_c_list=[]
var_rate_list=[]
for idx in range(len(X_tsfresh_filter.columns)):
    feature=X_tsfresh_filter.iloc[:,idx].values
    su_n_n_list.append(get_SU_norm_norm(feature,target_a))
    su_n_c_list.append(get_SU_norm_norm(feature,target_b))
    var_rate_list.append(get_var_rate(feature,range_list))
all_prop_list_raw = np.vstack([np.array(su_n_n_list),np.array(su_n_c_list),np.array(var_rate_list)]).T
all_prop_list = np.nan_to_num(all_prop_list_raw)


C:\Users\Administrator\Documents\yh.wang\2\utilities.py:265: RuntimeWarning: invalid value encountered in log2
  MI=np.log2(np.linalg.det(sigma_1)*np.linalg.det(sigma_2)/np.linalg.det(sigma_12))


In [19]:
object_func=1.5*all_prop_list[:,0]+1*all_prop_list[:,1]+0.5*all_prop_list[:,2]
len(object_func)

1140

In [37]:
col_idx=np.sort(object_func.argsort()[-450:])
col_idx

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,   10,   12,
         13,   14,   15,   16,   17,   18,   19,   22,   23,   24,   25,
         26,   27,   28,   29,   30,   31,   32,   33,   34,   35,   36,
         37,   38,   39,   40,   41,   42,   43,   44,   45,   46,   47,
         48,   49,   50,   52,   53,   54,   55,   56,   57,   58,   59,
         61,   62,   63,   64,   65,   66,   67,   68,   70,   71,   72,
         73,   74,   75,   76,   77,   78,   79,   80,   81,   82,   83,
         85,   86,   87,   89,   90,   91,   94,   95,   97,   98,   99,
        101,  102,  104,  105,  107,  109,  110,  111,  112,  113,  114,
        116,  118,  120,  121,  122,  127,  129,  131,  139,  140,  141,
        142,  143,  144,  146,  147,  154,  158,  160,  161,  172,  182,
        194,  195,  198,  204,  207,  214,  218,  223,  226,  234,  237,
        256,  261,  294,  306,  324,  326,  337,  349,  351,  354,  355,
        356,  366,  370,  378,  392,  437,  470,  4

In [38]:
X_tsfresh_filter_2 = X_tsfresh_filter.iloc[:,col_idx].copy()
X_tsfresh_filter_2

,"Ps30__linear_trend__attr_""rvalue""","T50__linear_trend__attr_""rvalue""","Ps30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","T50__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","Ps30__agg_autocorrelation__f_agg_""mean""__maxlag_40","Ps30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""min""","T30__linear_trend__attr_""rvalue""","T30__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","T50__agg_autocorrelation__f_agg_""mean""__maxlag_40","T50__agg_autocorrelation__f_agg_""median""__maxlag_40",...,BPR__skewness,phi__longest_strike_above_mean,"phi__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""",phi__index_mass_quantile__q_0.7,phi__index_mass_quantile__q_0.9,"phi__fft_coefficient__attr_""abs""__coeff_2",phi__index_mass_quantile__q_0.8,P30__index_mass_quantile__q_0.8,"phi__fft_coefficient__attr_""abs""__coeff_1",phi__autocorrelation__lag_9
1,0.036808,-0.045074,0.085754,-0.125201,-0.015258,-0.037778,-0.021656,-0.033845,-0.015560,-0.017653,...,0.354175,9.0,1.000000,0.693878,0.918367,2.738731,0.806122,0.806122,5.241730,0.132959
2,0.678206,0.593719,0.885443,0.876356,0.384595,0.830396,0.427227,0.727215,0.274177,0.257255,...,-0.196909,10.0,-0.868937,0.714876,0.909091,2.074138,0.814050,0.818182,5.335814,-0.004809
3,0.391805,0.377902,0.648412,0.732115,0.120158,0.481087,0.298577,0.651584,0.101254,0.118426,...,0.151547,20.0,0.971258,0.764706,0.932127,8.236927,0.850679,0.832579,13.604693,0.435131
4,0.512313,0.468434,0.752620,0.794643,0.226464,0.598739,0.377685,0.723493,0.173509,0.184361,...,0.053874,45.0,0.953149,0.776423,0.934959,11.171961,0.861789,0.841463,21.041474,0.568850
5,-0.065119,-0.071720,-0.132862,-0.110864,-0.024150,-0.177193,-0.123445,-0.267598,-0.013678,-0.011296,...,0.351869,5.0,-1.000000,0.676471,0.882353,0.889441,0.764706,0.808824,1.695958,0.066453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.540671,0.441477,0.848319,0.773979,0.241943,0.682978,0.256907,0.518524,0.136777,0.134703,...,-0.450039,9.0,-0.259754,0.709845,0.906736,0.470068,0.813472,0.823834,3.491504,0.053640
247,0.420526,0.361882,0.745867,0.714166,0.084149,0.615571,0.291753,0.577552,0.063121,0.083356,...,-0.734085,7.0,0.868305,0.706422,0.899083,1.071165,0.798165,0.807339,0.819272,0.035548
248,0.816956,0.730554,0.942471,0.924612,0.520916,0.879056,0.645477,0.893074,0.388455,0.409249,...,-0.125311,12.0,-0.869066,0.735751,0.917098,4.126003,0.829016,0.839378,6.822747,0.264636
249,0.153639,0.129724,0.273206,0.247850,-0.023117,0.308664,0.087300,0.190126,-0.015016,0.004923,...,0.424036,8.0,1.000000,0.753846,0.923077,1.136401,0.830769,0.846154,4.269818,-0.008270


# bfcf

In [39]:
msu_list = []
for idx in range(len(X_tsfresh_filter_2.columns)):
    feature=X_tsfresh_filter_2.iloc[:,idx].values
    msu_list.append(get_SU_norm_cls_norm(feature,target_b,target_a))
list_origin = np.nan_to_num(np.array(msu_list)).argsort()[::-1]
list_origin_origin = list_origin.copy()

C:\Users\Administrator\Documents\yh.wang\2\utilities.py:294: RuntimeWarning: divide by zero encountered in log2
  MI=np.log2(sigma_x)+np.log2(sigma_y2)-p_0*np.log2(Sigma_0)-p_1*np.log2(Sigma_1)


In [40]:
for count in range(len(list_origin)):
    if list_origin[count] != -1: 
        feature_ = X_tsfresh_filter_2.iloc[:,list_origin[count]].values
        for count_next in range(count+1,len(list_origin)):
            if list_origin[count_next] != -1: 
                feature = X_tsfresh_filter_2.iloc[:,list_origin[count_next]].values
                if get_SU_norm_cls_norm(feature_,target_b,target_a) < get_SU_norm_norm(feature_,feature):
                    list_origin[count_next] = -1

C:\Users\Administrator\Documents\yh.wang\2\utilities.py:265: RuntimeWarning: divide by zero encountered in double_scalars
  MI=np.log2(np.linalg.det(sigma_1)*np.linalg.det(sigma_2)/np.linalg.det(sigma_12))
C:\Users\Administrator\Documents\yh.wang\2\utilities.py:265: RuntimeWarning: invalid value encountered in log2
  MI=np.log2(np.linalg.det(sigma_1)*np.linalg.det(sigma_2)/np.linalg.det(sigma_12))


In [41]:
list_origin,list_origin_origin

(array([162, 163,  -1, 165, 166, 167, 168, 169,  -1, 171, 439, 441, 172,
        388,  -1, 448,  -1, 203, 226,  -1,  -1,  -1,  -1, 349,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1, 372,  -1, 192,  -1,  -1, 407,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 132,
         -1,  -1, 357,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 154,
         -1,  -1, 230,  -1, 411,  -1,  -1, 173, 380,  -1,  -1,  -1,  -1,
        148,  -1,  -1, 383,  -1,  -1,  -1,  -1, 378,  -1,  -1, 431,  -1,
         -1, 384,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1, 222,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 391,  -1,  -1,  -1, 386,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 184, 352,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 314,  -1, 424,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1

In [44]:
haha = np.array([162, 163,  -1, 165, 166, -1, 168, 169,  -1, 171, 439, 441, 172,
        388,  -1, 448,  -1, 203, 226,  -1,  -1,  -1,  -1, 349,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1, 372,  -1, 192,  -1,  -1, 407,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 132,
         -1,  -1, 357,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 154,
         -1,  -1, 230,  -1, 411,  -1,  -1, 173, 380,  -1,  -1,  -1,  -1,
        148,  -1,  -1, 383,  -1,  -1,  -1,  -1, 378,  -1,  -1, 431,  -1,
         -1, 384,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1, 222,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 391,  -1,  -1,  -1, 386,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 184, 352,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 314,  -1, 424,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1, 359,  -1,  -1,  -1, 347,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
        396, 350,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 313,  -1,
        449,  -1,  -1,  -1,  -1,  -1,  -1, 341,  -1,  -1, 426,  -1, 397,
         -1,  -1,  -1, 356,  -1,  -1, 395,  -1,  -1, 218,  -1, 444,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1, 435, 151,  -1,  -1,  -1,  -1,
         -1, 436,  -1, 329,  -1, 367, 419,  -1, 408,  -1,  -1,  -1,  -1,
        432,  -1, 434,  -1, 377, 231, 294,   0,  -1, 312,  -1,  17,  -1,
         -1,  -1,  -1, 442,  -1,  -1, 410,  -1, 392,  -1,  -1,  -1, 421,
         -1,  -1, 298,  -1, 437,  -1,  -1,  -1,  -1,  -1, 399,   6,  -1,
         -1, 106,  -1, 153,  -1,  19,  -1,  -1, 403,  -1,  -1,  -1,  -1,
         11,  -1,  -1,  -1,  -1,  50,  77,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 425,  84,  26,  -1,
        423,  -1,  65,  -1, 135,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1, 225,  34,  -1,  -1,
         -1,  -1,  -1,  -1,  31,  -1,  -1, 440,  -1,  -1,  -1,  -1,  -1,
         41,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  68,  -1,  -1,  -1,  -1,  -1,  97,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
        116,  -1,  -1,  -1,  -1,  98,  -1,  -1,  -1,  -1,  -1,  96,  81,
         -1,  -1,  -1,  43,  71,  -1,  -1, 113], dtype=np.int64)

In [45]:
col_idx_2 = np.sort(list_origin_origin[np.where(haha != -1)])
col_idx_2

array([  0,   6,  11,  17,  19,  26,  31,  34,  41,  43,  50,  65,  68,
        71,  77,  81,  84,  96,  97,  98, 106, 113, 116, 132, 135, 148,
       151, 153, 154, 162, 163, 165, 166, 168, 169, 171, 172, 173, 184,
       192, 203, 218, 222, 225, 226, 230, 231, 294, 298, 312, 313, 314,
       329, 341, 347, 349, 350, 352, 356, 357, 359, 367, 372, 377, 378,
       380, 383, 384, 386, 388, 391, 392, 395, 396, 397, 399, 403, 407,
       408, 410, 411, 419, 421, 423, 424, 425, 426, 431, 432, 434, 435,
       436, 437, 439, 440, 441, 442, 444, 448, 449], dtype=int64)

# 将结果装载到tsfresh中

In [46]:
# kind_to_fc_parameters_train=tsfresh.feature_extraction.settings.from_columns(X_tsfresh_filter.iloc[:,col_idx])
kind_to_fc_parameters_train=tsfresh.feature_extraction.settings.from_columns(X_tsfresh_filter_2.iloc[:,col_idx_2])
# save_dict(kind_to_fc_parameters_train,"./numpy/kind_to_fc_parameters_0322")
save_dict(kind_to_fc_parameters_train,"./train004/numpy/kind_to_fc_parameters_0420")

In [49]:
dataframe_list=[]
for idx in tqdm(range(len(train_dataset_expend))):
    dataframe_list.append(get_dataframe_ofcuttedsequence_new(train_dataset_expend,idx,
                                                             attribute[2:]))
train_dataset_expend_dataframe=pd.concat(dataframe_list).reset_index(drop=True)
# train_dataset_expend_dataframe.to_csv("./numpy/train_dataset_expend_dataframe.csv")
extracted_features_enhanced = extract_features(train_dataset_expend_dataframe,
                                              column_id="Unit",
                                              column_sort="Time",
                                             kind_to_fc_parameters =kind_to_fc_parameters_train)
extracted_features_enhanced=impute(extracted_features_enhanced)

  0%|          | 0/48799 [00:00<?, ?it/s]

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 40/40 [05:51<00:00,  8.78s/it]


In [50]:
dataframe_test_list = []
for idx in tqdm(range(len(test_dataset))):
    dataframe_test_list.append(get_dataframe_ofcuttedsequence_new(test_dataset, idx,attribute[2:]))
test_dataset_dataframe = pd.concat(dataframe_test_list).reset_index(drop=True)
test_extracted_features = extract_features(test_dataset_dataframe,
                                             column_id="Unit",
                                             column_sort="Time",
                                             kind_to_fc_parameters=kind_to_fc_parameters_train)
test_extracted_features = impute(test_extracted_features)

  0%|          | 0/248 [00:00<?, ?it/s]

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.78it/s]


In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1,1))
features_extracted_minmax=scaler.fit_transform(np.vstack([extracted_features_enhanced.values,
                                                          test_extracted_features.values]))
# train_extracted_minmax,test_extracted_minmax=np.vsplit(features_extracted_minmax,np.array([24020]))
train_extracted_minmax,test_extracted_minmax=np.vsplit(features_extracted_minmax,np.array([48799]))

In [52]:
np.save("./train004/numpy/train_extracted_minmax_train_0419.npy", train_extracted_minmax)
np.save("./train004/numpy/test_extracted_minmax_train_0419.npy", test_extracted_minmax)

In [53]:
train_extracted_minmax.shape

(48799, 100)

In [ ]:
# 